In [ ]:
import torch
import torch
from diffusers import StableDiffusionPipeline, DDPMScheduler, AutoencoderKL
from PIL import Image
import sys
import os

import numpy as np
noise_scheduler = DDPMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    steps_offset=1,
)

In [ ]:
noise_scheduler

In [ ]:
import torch
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image
import sys
import os
sys.path.append(r'C:\Users\31878\Desktop\PoseCtrl-main')
sys.path.append(r'C:\Users\31878\Desktop\PoseCtrl-main\poseCtrl')
from poseCtrl.models.pose_adaptor import VPmatrixPoints, ImageProjModel
from poseCtrl.models.attention_processor import AttnProcessor, PoseAttnProcessor
from poseCtrl.data.dataset import CustomDataset, load_base_points
from poseCtrl.models.posectrl import PoseCtrl,PoseCtrlV1
import numpy as np

vae_model_path = "stabilityai/sd-vae-ft-mse"
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)
base_model_path = r"C:\Users\31878\Desktop\model"
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)


In [ ]:
pipe.unet

In [ ]:
unet_sd = pipe.unet.state_dict()
attn_procs = {}
for name in pipe.unet.attn_processors.keys():
        cross_attention_dim = None if name.endswith("attn1.processor") else pipe.unet.config.cross_attention_dim

        if name.startswith("mid_block"):
            hidden_size = pipe.unet.config.block_out_channels[-1]
        elif name.startswith("up_blocks"):
            block_id = int(name[len("up_blocks.")])
            hidden_size = list(reversed(pipe.unet.config.block_out_channels))[block_id]
        elif name.startswith("down_blocks"):
            block_id = int(name[len("down_blocks.")])
            print(block_id)
            hidden_size = pipe.unet.config.block_out_channels[block_id]
            # hidden_size = pipe.unet.config.block_in_channels[block_id]
            print(hidden_size)



In [ ]:
pipe.unet.down_blocks

In [ ]:
from copyUnet import *
base_model_path = r"C:\Users\31878\Desktop\model"
unet=CopyUnetDecoder.from_pretrained(base_model_path, subfolder="unet")


In [ ]:
unet

In [1]:
from copyUnet import *

from pathlib import Path
import torch.nn as nn
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import ProjectConfiguration
from diffusers import AutoencoderKL, DDPMScheduler, UNet2DConditionModel
from transformers import CLIPTextModel, CLIPTokenizer, CLIPVisionModelWithProjection, CLIPProcessor
import sys
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipelineLegacy, DDIMScheduler, AutoencoderKL
from PIL import Image
import numpy as np

device = "cpu"

base_model_path = r"C:\Users\31878\Desktop\model"
noise_scheduler = DDPMScheduler.from_pretrained(base_model_path, subfolder="scheduler")
tokenizer = CLIPTokenizer.from_pretrained(base_model_path, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(base_model_path, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(base_model_path, subfolder="vae")
# unet = UNet2DConditionModel.from_pretrained(base_model_path, subfolder="unet")
# image_encoder = CLIPVisionModelWithProjection.from_pretrained(base_model_path)
processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")
# freeze parameters of models to save more memory

vae.requires_grad_(False)
text_encoder.requires_grad_(False)

vae.to(device)
text_encoder.to(device)

unet=CopyUnetDecoder()
unet.to(device)
unet.requires_grad_(False)
unet.CopyUp_blocks.requires_grad_(True)
unet.to(device)
# image_encoder.requires_grad_(False)

C:\Users\31878\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\31878\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


copyUnet


CopyUnetDecoder(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0): Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features=320, out_features=320,

In [3]:
from poseCtrl.data.dataset import CustomDataset, load_base_points
import torch.nn as nn
import torch
import torch.nn.functional as F
train_dataset = CustomDataset(r'C:\Users\31878\Desktop\pic')
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=4,
    num_workers=0,
)
for step, batch in enumerate(train_dataloader):
    latents = vae.encode(batch["image"].to(device)).latent_dist.sample()
    latents = latents * vae.config.scaling_factor

    # Sample noise that we'll add to the latents
    noise = torch.randn_like(latents)
    bsz = latents.shape[0]
    # Sample a random timestep for each image
    timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bsz,), device=latents.device)
    min_step = 10
    timesteps = timesteps.long()

    # Add noise to the latents according to the noise magnitude at each timestep
    # (this is the forward diffusion process)
    noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

    text_input_ids = tokenizer(
            "girl",
            max_length=tokenizer.model_max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        ).input_ids.to(device) 
    encoder_hidden_states = text_encoder(text_input_ids)[0]
    encoder_hidden_states = encoder_hidden_states.repeat(4, 1, 1) 

    print("noisy_latents size")
    print(noisy_latents.size())
    print("timesteps size")
    print(timesteps.size())
    print("encoder_hidden_states size")
    print(encoder_hidden_states.size())

    # noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states)

C:\Users\31878\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\diffusers\configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


noisy_latents size
torch.Size([4, 4, 64, 64])
timesteps size
torch.Size([4])
encoder_hidden_states size
torch.Size([4, 77, 768])


KeyboardInterrupt: 

In [2]:
import torch

# 生成 noisy_latents 数据
noisy_latents = torch.randn(4, 4, 64, 64).to(device)

# 生成 timesteps 数据
timesteps = torch.randint(0, 1000, (4,)).to(device)

# 生成 encoder_hidden_states 数据
encoder_hidden_states = torch.randn(4, 77, 768).to(device)

# 打印数据的形状
print("noisy_latents size:", noisy_latents.size())
print("timesteps size:", timesteps.size())
print("encoder_hidden_states size:", encoder_hidden_states.size())

noisy_latents size: torch.Size([4, 4, 64, 64])
timesteps size: torch.Size([4])
encoder_hidden_states size: torch.Size([4, 77, 768])


In [3]:
noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states)


torch.Size([4, 320, 64, 64])
execute
sample.size
torch.Size([4, 320, 64, 64])
copySample.size
torch.Size([4, 320, 64, 64])
lerpAlphaTest


In [4]:
Ori_unet = UNet2DConditionModel.from_pretrained(base_model_path, subfolder="unet")
Ori_unet.to(device)
Ori_noise_pred= Ori_unet(noisy_latents, timesteps, encoder_hidden_states=encoder_hidden_states)

In [5]:
def count_parameters(model):
    total_params = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            total_params += param.numel()
            print(f"Parameter: {name}, Size: {param.size()}, Num Elements: {param.numel()}")
    return total_params

total_params = count_parameters(Ori_unet)
print(f"Total trainable parameters: {total_params}")

NameError: name 'Ori_unet' is not defined

In [5]:
def count_parameters(model):
    total_params = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            total_params += param.numel()
            print(f"Parameter: {name}, Size: {param.size()}, Num Elements: {param.numel()}")
    return total_params

total_params = count_parameters(unet)
print(f"Total trainable parameters: {total_params}")

Parameter: CopyUp_blocks.0.resnets.0.norm1.weight, Size: torch.Size([2560]), Num Elements: 2560
Parameter: CopyUp_blocks.0.resnets.0.norm1.bias, Size: torch.Size([2560]), Num Elements: 2560
Parameter: CopyUp_blocks.0.resnets.0.conv1.weight, Size: torch.Size([1280, 2560, 3, 3]), Num Elements: 29491200
Parameter: CopyUp_blocks.0.resnets.0.conv1.bias, Size: torch.Size([1280]), Num Elements: 1280
Parameter: CopyUp_blocks.0.resnets.0.time_emb_proj.weight, Size: torch.Size([1280, 1280]), Num Elements: 1638400
Parameter: CopyUp_blocks.0.resnets.0.time_emb_proj.bias, Size: torch.Size([1280]), Num Elements: 1280
Parameter: CopyUp_blocks.0.resnets.0.norm2.weight, Size: torch.Size([1280]), Num Elements: 1280
Parameter: CopyUp_blocks.0.resnets.0.norm2.bias, Size: torch.Size([1280]), Num Elements: 1280
Parameter: CopyUp_blocks.0.resnets.0.conv2.weight, Size: torch.Size([1280, 1280, 3, 3]), Num Elements: 14745600
Parameter: CopyUp_blocks.0.resnets.0.conv2.bias, Size: torch.Size([1280]), Num Elements